# Timeline.plot testing


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import mesonic

In [ ]:
import pyamapping as pam
import math
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 2]
plt.rcParams['figure.dpi'] = 100

In [ ]:
context = mesonic.create_context()

In [ ]:
context.timeline

In [ ]:
s1m = context.synths.create("s1")
s1i = context.synths.create("s1", mutable=False)
s2 = context.synths.create("s2", track=1)

In [ ]:
context.synths

In [ ]:
context.clear() 
with context.at(0.3):
    s2.start(freq=300, amp=0.1, pan=1)
for t in range(4,10):
    with context.at(t/10):  
        s2.freq = 100 * t
        s2.amp += 0.1
        s2.pan = -1 * s2.pan.value
with context.at((t+1)/10): 
    s2.stop()

context.time = 1.25
s1m.start(dur=2, pan=1, freq=450)  # parameters can be provided by keyword
for i in range(1,5):
    context.time += 0.25
    s1m.pan *= -1  # it is also possible to use /= , += , -=    
    if i % 2 == 0:
        print("stop")
        context.time += 1
        s1m.stop()
        context.time += 1
        s1m.start()
context.time = None 

for i in range(5):
    with context.at(3+i/4):
        s1i.start(dur=1.0, freq=100 * (i+1))

In [ ]:
context.timeline

In [ ]:
context.timeline.plot_new()

In [ ]:
context.timeline

In [ ]:
plt.figure()
context.timeline.plot_new(offset="amp", width="freq")
#plt.semilogy()

In [ ]:
context.timeline.plot_new(offset="freq", width="pan")

In [ ]:
context.timeline.plot_new(offset="pan", width="freq")

In [ ]:
def safe_amp_to_db(amp):
    return np.where(amp > 0, pam.amp_to_db(amp), -90)


In [ ]:
safe_amp_to_db(-1)

In [ ]:
from mesonic.events import SynthEventType
from mesonic.timeline import SynthEventPseudoStop

In [ ]:
def print_events(events, etype):
    print(len(events[etype]), [t[0] for t in events[etype]])
    
for synth, events in context.timeline._check_consistency().items():
    print()
    print(synth)
    print_events(events, SynthEventType.START)
    print_events(events, SynthEventType.SET)
    print_events(events, SynthEventType.STOP)
    assert len(events[SynthEventType.START]) == len(events[SynthEventType.STOP])

In [ ]:
s1i._param_values()

In [ ]:
val = 2.5

li = list(range(5))
le = list(range(1,6))
print(li)
print(le)

iidx = 0
for val in [0.5, 1.5,3.5,4.5]:
    print(val)
    while iidx < len(li) and li[iidx] < val:
        iidx += 1

    print(val, iidx)
    li.insert(iidx, val)
    le.insert(iidx-1, val)
    print(li)
    print(le)

In [ ]:
synth_plot_info = context.timeline._create_segments()

In [ ]:
for synth, values in synth_plot_info.items():
    print(synth)
    print(synth.mutable)
    for etype, events in values["events"].items():
        print(etype, [t[0] for t in events])
    for key, vs in values["segments"].items():
        print(key, "   \t", vs)

In [ ]:
fig = plt.figure()
cmap = plt.get_cmap("Set1")
axes = fig.add_subplot(1, 1, 1)

for synth in synth_plot_info.keys():
    segments = synth_plot_info[synth]["segments"]
    quiver = axes.quiver(
        segments["begins"],
        segments["offsets"],
        np.array(segments["ends"]) - np.array(segments["begins"]),
        0.0,
        angles=0,
        scale=1,
        scale_units="x",
        headwidth=0,
        headlength=0,
        headaxislength=0,
        linewidth=segments["widths"],
    )
    quiver.set_linewidths(segments["widths"])

In [ ]:
list(zip(zip(segments["begins"], segments["offsets"]), zip(segments["ends"], segments["offsets"])))

In [ ]:
fig = plt.figure()
cmap = plt.get_cmap("Set1")
axes = fig.add_subplot(1, 1, 1)
from matplotlib.collections import LineCollection

x_min, x_max = 0, 0
y_min, y_max = 0, 0

for synth in synth_plot_info.keys():
    segments = synth_plot_info[synth]["segments"]
    s = np.array(list(zip(
        list(zip(segments["begins"], segments["offsets"])),
        list(zip(segments["ends"], segments["offsets"])),
    )))
    lc = LineCollection(
        s,
        linewidth=segments["widths"],
    )
    axes.add_collection(lc)

    x_min = min(np.min(segments["begins"]), x_min)
    x_max = max(np.max(segments["ends"]), x_max)
    y_min = min(np.min(segments["offsets"]), y_min)
    y_max = max(np.max(segments["offsets"]), y_max)

x_lim = (x_max-x_min) * 0.1
axes.set_xlim(x_min-x_lim,x_max+x_lim)
y_lim = (y_max-y_min) * 0.1
axes.set_ylim(y_min-y_lim,y_max+y_lim)
axes.grid()
plt.show()

In [ ]:
s.shape

In [ ]:
synth_plot_info[s2]["segments"]

In [ ]:
synth_plot_info[s1i]["segments"]

In [ ]:
synth_plot_info[s1m]["segments"]

In [ ]:
x = np.arange(0,128)
plt.plot(x, np.log(pam.midicps(x)))

In [ ]:
x = np.arange(0,128)
plt.plot(x, (pam.midicps(x)))

In [ ]:
x = np.arange(0,128)
plt.plot(x, pam.midicps(x))
plt.yscale("log")

In [ ]:
s1m.freq.bounds = (300,1000)
s1i.freq.bounds = (100,1000)
s2.freq.bounds = (300,1000)

In [ ]:
s1m.pan.bounds, s1i.pan.bounds, s2.pan.bounds

In [ ]:
context.timeline.plot()

In [ ]:
for s in [s1m, s1i, s2]:
    s1m.pan.bounds = -1,1

In [ ]:
context.timeline.plot("pan")

In [ ]:
# close the context.
context.close()
# if all contexts are closed the backend should also exit.